# Libraries

In [1]:
import pandas as pd
import pandas.io.sql as sqlio
import psycopg2 as ps

# 0. Connection to Postgre

In [2]:
con = ps.connect(dbname="fintech",
                   user = "postgres",
                   password = "lolita",
                   host = "localhost",
                   port = "5432")

In [3]:
sql = """select * from fintech_user_f"""

In [4]:
df = sqlio.read_sql_query(sql, con)

C:\Users\luisf\AppData\Local\Temp\ipykernel_21084\2699896095.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = sqlio.read_sql_query(sql, con)


In [5]:
df.head()

,user_id,churn,age,housing,credit_score,deposits,withdrawal,purchases_partners,purchases,cc_taken,...,registered_phones,payment_type,waiting_4_loan,cancelled_loan,received_loan,rejected_loan,zodiac_sign,rewards_earned,reward_rate,is_referred
0,1,False,21.0,R,577.0,48,4,52,45,0,...,2,Semi-Monthly,False,False,False,False,Pisces,56.0,1.87,False
1,4,False,35.0,R,566.0,15,0,73,15,0,...,0,Semi-Monthly,False,False,False,False,na,49.0,1.63,False
2,8,True,31.0,na,519.0,0,0,0,0,0,...,0,Bi-Weekly,False,False,False,False,Virgo,18.0,0.60,True
3,9,False,26.0,na,NaN,0,0,30,0,0,...,2,Weekly,False,False,False,False,Sagittarius,23.0,0.77,False
4,10,False,33.0,R,558.0,0,0,76,0,0,...,0,Bi-Weekly,False,False,False,False,Leo,45.0,1.50,True


---
---
# 1.Cleaning and formatting

## 1.0 Null values
### 1.0.0 NaNs

In [6]:
df.isna().sum()

user_id                    0
churn                      0
age                        4
housing                    0
credit_score            7369
deposits                   0
withdrawal                 0
purchases_partners         0
purchases                  0
cc_taken                   0
cc_recommended             0
cc_disliked                0
cc_liked                   0
cc_application_begin       0
app_downloaded             0
web_user                   0
app_web_user               0
ios_user                   0
android_user               0
registered_phones          0
payment_type               0
waiting_4_loan             0
cancelled_loan             0
received_loan              0
rejected_loan              0
zodiac_sign                0
rewards_earned          2996
reward_rate                0
is_referred                0
dtype: int64

In [7]:
print("NaNs in credit_score represent",df[df.credit_score.isna()].shape[0]/df.shape[0]*100, "% of total rows.")
print("NaNs in rewards represent",df[df.rewards_earned.isna()].shape[0]/df.shape[0]*100, "% of total rows.")

NaNs in credit_score represent 29.78456812578311 % of total rows.
NaNs in rewards represent 12.109453942847905 % of total rows.


_Filling rewards with 0s_

In [8]:
df.rewards_earned.unique()

array([ 56.,  49.,  18.,  23.,  45.,  60.,  61.,  53.,  37.,  34.,  12.,
        13.,  47.,  40.,  26.,  35.,  25.,  10.,  65.,  29.,  22.,  75.,
        44.,  63.,  32.,  51.,  41.,   5.,  21.,   6.,  67.,   2.,  68.,
        52.,  84.,   9.,  59.,  nan,  64.,  15.,  11.,  14.,  54.,  46.,
        24.,  48.,  33.,   4.,  58.,  43.,  19.,   1.,  50.,  17.,   3.,
        57.,  28.,  69.,   7.,  16.,  55.,  30.,  39.,  31.,  66.,  27.,
         8.,  42.,  72.,  36.,  76.,  62.,  20.,  38.,  71.,  90.,  80.,
        78.,  77.,  81.,  70.,  74.,  73.,  87.,  79.,  85.,  86.,  83.,
       114.,  99.,  82.,  88.])

In [9]:
df.rewards_earned = df.rewards_earned.fillna(0.)

_Putting mean in credit_score NaN values_

In [10]:
df.credit_score = df.credit_score.fillna(df.credit_score.mean())

In [11]:
print("NaNs in credit_score represent",df[df.credit_score.isna()].shape[0]/df.shape[0]*100, "% of total rows.")
print("NaNs in rewards represent",df[df.rewards_earned.isna()].shape[0]/df.shape[0]*100, "% of total rows.")

NaNs in credit_score represent 0.0 % of total rows.
NaNs in rewards represent 0.0 % of total rows.


### 1.1.1 NaNs in str format

In [24]:
for c in df.columns:
    print(c,df[c].unique())

user_id [    1     8     9 ... 69653 69655 69658]
churn [False  True]
age [21. 31. 26. 33. 54. 27. 28. 42. 25. 38. 24. 44. 30. 41. 36. 49. 22. 65.
 52. 45. 29. 35. 18. 51. 32. 43. 34. 19. 40. 48. 46. 23. 56. 37. 20. 39.
 66. 62. 47. 75. 60. 53. 61. 50. 55. 70. 64. 58. 63. 57. 59. 69. 84. 91.
 73. 17. 78. 89. 72. 68. 85. 67. 77. 74. 71. 80. 82. 83. 87. 81. 76. 79.]
credit_score [577.         519.         542.51559982 558.         559.
 554.         501.         495.         500.         533.
 532.         557.         590.         494.         550.
 562.         609.         481.         655.         364.
 525.         569.         513.         473.         555.
 649.         528.         526.         596.         620.
 454.         538.         647.         506.         490.
 613.         530.         461.         413.         588.
 581.         587.         535.         687.         491.
 472.         605.         570.         597.         545.
 638.         539.         531.         

In [12]:
# Dont need this column
df.housing.unique()

array(['R', 'na', 'O'], dtype=object)

In [13]:
df.drop(columns = 'housing', inplace = True)

In [14]:
df.zodiac_sign.unique()

array(['Pisces', 'na', 'Virgo', 'Sagittarius', 'Leo', 'Capricorn',
       'Gemini', 'Cancer', 'Scorpio', 'Libra', 'Taurus', 'Aquarius',
       'Aries'], dtype=object)

In [15]:
print("na values in zodiac_sign represent",df[df.zodiac_sign == 'na'].shape[0]/df.shape[0]*100, "% of total rows.")

na values in zodiac_sign represent 7.958449537205449 % of total rows.


In [16]:
df = df[df.zodiac_sign != 'na'].copy()

In [21]:
# too much
print("na values in payment_type represent",df[df.payment_type == 'na'].shape[0]/df.shape[0]*100, "% of total rows.")

na values in payment_type represent 14.531002986123308 % of total rows.


In [25]:
df.payment_type = df.payment_type.fillna("Monthly")